# --- Day 14 Regolith Reservoir ---

https://adventofcode.com/2022/day/14

## Get Input Data

In [1]:
def get_data(filename):
    """Get input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    rock_lines : list of list of tuples
        Each element is a line from the file, containing a list of tuples; 
        each tuple is a point on a line segment
    map_dims : dict
        Contains the 'top', 'right', 'bottom', and 'left' boundry of the map scan.
    """

    with open(f'../inputs/{filename}.txt') as file:
        rock_lines = [line.strip().split(' -> ') for line in file]

    map_dims = {'top': 0, 'right': 500, 'bottom': 0, 'left':500}

    for line in rock_lines:
        for i, pos in enumerate(line):
            
            # Create map dimensions
            x, y = pos.split(',')
            map_dims['top'] = min(map_dims['top'], int(y))
            map_dims['right'] = max(map_dims['right'], int(x))
            map_dims['bottom'] = max(map_dims['bottom'], int(y))
            map_dims['left'] = min(map_dims['left'], int(x))
            
            # Convert line points to tuple of integers
            line[i] = (int(x), int(y))

    return rock_lines, map_dims

In [2]:
test_rock_lines, test_map_dims = get_data('test_cave_scan')
print(test_rock_lines)
print(test_map_dims)

[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]]
{'top': 0, 'right': 503, 'bottom': 9, 'left': 494}


## Part 1
---

In [3]:
import numpy as np

In [4]:
def make_cave_map(rock_lines):
    """Make a map of the cave, based on the rock lines data.

    Parameters
    ----------
    rock_lines : list of list of tuples
        Each element is a line from the file, containing a list of tuples; 
        each tuple is a point on a line segment

    Returns
    -------
    cave_map: np.ndarray
        All zeros where there is air, and 1s where there are rocks
    """

    # Make an arbitarily large map...
    cave_map = np.zeros((1000, 1000), dtype=np.int8)

    for line in rock_lines:
        for i in range(1, len(line)):
            x1, y1 = line[i-1][0], line[i-1][1]
            x2, y2 = line[i][0], line[i][1]

            for x in range(min(x1, x2), max(x1, x2) + 1):
                for y in range(min(y1, y2), max(y1, y2) + 1):
                    # map coordinates are (x, y), but numpy arrays are (row, col),
                    # so need to reverse x, y
                    cave_map[y, x] = 1

    return cave_map

In [5]:
test_rock_lines, test_map_dims = get_data('test_cave_scan')
test_cave_map = make_cave_map(test_rock_lines)
test_cave_map[0:10, 494:504]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]], dtype=int8)

In [6]:
possible_moves = {'down' : (1, 0), 'down_left' : (1, -1), 'down_right' : (1, 1)}

In [7]:
def check_not_blocked(cave_map, curr_pos, possible_moves):
    """Check that the positions below the current position are not blocked.

    Parameters
    ----------
    cave_map : np.ndarray
        Arbitrarily large map of the cave --> 1 = rock, 0 = air
    curr_pos : tuple
        Current position in the cave
    possible_moves : dict
        Contains tuples for three downward moves from current position

    Returns
    -------
    not_blocked : dict
        Three keys: 'down', 'down_left', and 'down_right' with boolean values for whether the cell
        in those positions relative to the current position in the cave is blocked by either rock (1s)
        or sand (8s)
    """

    not_blocked = {}

    # Need to "not" the boolean value, because blocked means there's a value of 1 in the cell,
    # which is truthy
    not_blocked['down'] = not bool(cave_map[tuple(map(sum, zip(curr_pos, possible_moves['down'])))])
    not_blocked['down_left'] = not bool(cave_map[tuple(map(sum, zip(curr_pos, possible_moves['down_left'])))])
    not_blocked['down_right'] = not bool(cave_map[tuple(map(sum, zip(curr_pos, possible_moves['down_right'])))])

    return not_blocked

In [8]:
def off_the_map(map_dims, curr_pos):
    """_summary_

    Parameters
    ----------
    map_dims : dict
        Dimensions of the cave map
    curr_pos : tuple
        Current position in the cave

    Returns
    -------
    Bool
        True of off the map, False otherwise
    """
    
    if (curr_pos[0] > map_dims['bottom'] or 
        curr_pos[1] < map_dims['left'] or 
        curr_pos[1] > map_dims['right']):
        return True
    else:
        return False

In [9]:
def settle(cave_map, map_dims, possible_moves):
    """Move sand through the cave until it settles.

    Parameters
    ----------
    cave_map : np.ndarray
        Arbitrarily large map of the cave -> 1s = rock, 0s = air, 8s = sand
    map_dims : dict
        Contains edge values of the cave map dimensions
    possible_moves : dict
        Contains tuples for three downward moves from current position

    Returns
    -------
    cave_map : np.ndarray
        Updated cave map with sand in new location
    next_pos : tuple or None
        Position of last location sand settled; None when sand starts falling off side of the map
    """

    SOURCE = (0, 500)
    next_pos = SOURCE

    while True:
        not_blocked = check_not_blocked(cave_map, next_pos, possible_moves)

        if not_blocked['down']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down'])))
        elif not_blocked['down_left']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down_left'])))
        elif not_blocked['down_right']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down_right'])))
        else:
            cave_map[next_pos] = 8  # Arbitrary postitive value for debugging...
            break

        # Off the map...
        if off_the_map(map_dims, next_pos):
            next_pos = None
            break

    return cave_map, next_pos

In [10]:
def solve(filename):
    """Solve part 1.

    Count how many units of sand fall before sand starts falling off the edge of the map.

    Parameters
    ----------
    filename : str
        Name of file containing input data of rock lines data

    Returns
    -------
    num_sand_units
        Count of number of units of sand released before they fall off the edge of the map
    """

    rock_lines, map_dims = get_data(filename)
    cave_map = make_cave_map(rock_lines)

    num_sand_units = 0

    while True:
        new_cave_map, last_pos = settle(cave_map, map_dims, possible_moves)
        if last_pos == None:
            return num_sand_units
        num_sand_units += 1

### Run on Test Data

In [11]:
solve('test_cave_scan') == 24

True

### Run on Input Data

In [12]:
solve('cave_scan')

768

## Part 2
---

In [13]:
def add_floor(cave_map, map_dims):
    """Add a floor to the cave map.
    The floor is at 2 units "above" the max value for the "bottom"

    Parameters
    ----------
    cave_map : np.ndarray
        Arbitrarily large map of the cave
    map_dims : dict
        Contains edge values of the cave map dimensions

    Returns
    -------
    cave_map : np.ndarray
        An updated cave map
    """

    _floor = map_dims['bottom'] + 2
    cave_map[_floor, :] = 1

    return cave_map

In [14]:
def settle2(cave_map, map_dims, possible_moves):
    """Move sand through the cave until it settles.

    Parameters
    ----------
    cave_map : np.ndarray
        Arbitrarily large map of the cave -> 1s = rock, 0s = air, 8s = sand
    map_dims : dict
        Contains edge values of the cave map dimensions
    possible_moves : dict
        Contains tuples for three downward moves from current position

    Returns
    -------
    cave_map : np.ndarray
        Updated cave map with sand in new location
    next_pos : tuple or None
        Position of last location sand settled; None when sand starts falling off side of the map
    """

    SOURCE = (0, 500)
    next_pos = SOURCE

    while True:
        not_blocked = check_not_blocked(cave_map, next_pos, possible_moves)

        if not_blocked['down']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down'])))
        elif not_blocked['down_left']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down_left'])))
        elif not_blocked['down_right']:
            next_pos = tuple(map(sum, zip(next_pos, possible_moves['down_right'])))
        else:
            cave_map[next_pos] = 8
            break

    return cave_map, next_pos

In [15]:
def solve2(filename):
    """Solve part 2.

    Count how many units of sand fall before the last unit of sand is at position (0, 500).

    Parameters
    ----------
    filename : str
        Name of file containing input data of rock lines data

    Returns
    -------
    num_sand_units
        Count of number of units of sand released
    """

    rock_lines, map_dims = get_data(filename)
    cave_map = make_cave_map(rock_lines)

    cave_map = add_floor(cave_map, map_dims)
    # print(cave_map[0:13, 494:504])

    num_sand_units = 0

    while True:
        new_cave_map, last_pos = settle2(cave_map, map_dims, possible_moves)
        if last_pos == (0, 500):
            return num_sand_units + 1
        num_sand_units += 1

### Run on Test Data

In [16]:
solve2('test_cave_scan') == 93

True

### Run on Input Data

In [17]:
solve2('cave_scan')

26686